# Manejando datos faltantes (missing data)

La diferencia entre los datos que vemos en la mayoría de ejemplos y los datos que nos podemos encontrar en los casos reales que vemos en una empresa reside en la naturaleza de los propios datos, ya que en el mundo real rara vez nos encontramos con datos homogéneos que estén limpios y preparados para ser utilizados. En particular, a muchos datasets les faltará algún que otro dato.

En esta sección, veremos algunas consideraciones generales respecto a los datos faltantes, cómo tratarlos y algunas herramientas integradas de Pandas para manejar este tipo de datos en Python.

De ahora en adelante, veremos que los valores nulos o datos faltantes se representan como *null*, *NaN*, o *NA*.

## Trade-off en las convenciones de datos faltantes

Principalmente, existen 2 estrategias para detectar datos faltantes: usar una máscara que indique globalmente los valores faltantes, o elegir un valor centinela que indique que no se dispone de cierto dato (que visto desde fuera será un valor ya elegido por el "creador" del dataset").

En primer enfoque, el enfoque de enmascaramiento, se basa en detectar los datos faltantes mediante el uso de una matriz booleana que funcionará como máscara, o a partir de otra columna que indique localmente el estado nulo de un valor.

Por otra parte, tenemos el enfoque centinela, donde el valor principal (o valor centinela) podría ser una convención específica de datos, como indicar un valor entero faltante con -9999 o algún otro patrón más específico. Sin embargo, también es típica una convención más global, como indicar los valores faltantes con *NaN* (Not a Number), un valor especial que forma parte de la especificación de coma flotante IEEE y que veremos más adelante.

Ninguno de estos enfoques está exento de compensaciones:
  - El uso de la máscara requiere la asignación de una matriz booleana adicional, lo que agrega gastos generales tanto en almacenamiento como en cálculo.
  - Un valor centinela reduce el rango de valores válidos que se pueden representar y puede requerir lógica adicional (a menudo no optimizada) en la aritmética de CPU y GPU. Los valores especiales comunes como NaN no están disponibles para todos los tipos de datos.

Como en la mayoría de los casos en los que no existe una opción óptima universal, los diferentes lenguajes y sistemas utilizan diferentes convenciones.

## Datos faltantes en Pandas

La forma en que Pandas maneja los valores faltantes está limitada por su dependencia del paquete NumPy, que no tiene nada incorporado para representar valores NA que no sea de tipo punto flotante.

NumPy admite muchos tipos básicos de datos, donde los enteros pueden ser representados por hasta 14 tipos de datos básicos diferentes, una vez que se tienen en cuenta las precisiones disponibles, la firma y la "endianidad" (ordne de transmisión de bits) de la codificación. Por ello, reservar una combinación de bits para hacer referencia al valor nulo daría lugar a una sobrecarga difícil de manejar, y que probablemente requeriría una nueva versión del paquete NumPy. Además, para los tipos de datos más pequeños (como enteros de 8 bits), sacrificar un bit para usarlo como máscara reducirá significativamente el rango de valores que puede representar.

Además, NumPy tiene soporte para matrices enmascaradas, es decir, matrices que tienen una matriz de máscara booleana separada adjunta para marcar datos como "buenos" o "malos", es decir, como datos disponibles o faltantes. Por su parte, Pandas podría haber recreado algo semejante, sin embargo, la sobrecarga que supondría tanto en almacenamiento y computación como en mantenimiento de código, hace que sea una elección poco atractiva.

Con estas limitaciones en mente, Pandas eligió usar la estrategia del valor centinela para los datos faltantes, y dos valores nulos de Python ya existentes: el valor especial de punto flotante ``NaN`` y el objeto Python ``None``. Como hemos comentado anteriormente, esta elección trae consigo algunos efectos secundarios, pero que, como veremos en este notebook, supondrá un buen compromiso en la mayoría de los casos de interés.

### ``None``: el representante "pitónico" (pythonic) para los datos faltantes

El primer valor centinela utilizado por Pandas es ``None``, un objeto único de Python que se usa normalmente para los datos faltantes en el código Python.
Debido a que es un objeto de Python, ``None`` no se puede usar en ninguna matriz arbitraria de NumPy o Pandas, solo en matrices con el tipo de datos ``object`` (es decir, matrices de objetos de Python):

In [6]:
import numpy as np
import pandas as pd

In [16]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

Si nos fijamos en el tipo del array de Numpy, vemos que indica ``dtype = object``, lo que significa que la mejor representación de tipo común que NumPy podría inferir para el contenido de la matriz es que son objetos de Python.

Si bien este tipo de matriz es útil para algunos casos de uso, cualquier operación con los datos se realizará a nivel de Python, lo que supondrá mayor sobrecarga que las operaciones eficientes que se utilizan para las matrices con tipos nativos.


Para demostrar esto, haremos uso de un comando mágico propio de los notebooks (no es una función de Python pero lo podremos utilizar aquí) que nos permitirá realizar una acción repetidas veces y promediar el tiempo que ha tardado en ejecutarse. Este comando es ``%timeit``, que repetirá n veces la línea que se escriba a continuación (donde n lo decidirá automáticamente) y nos mostrará el análisis de lo que ha tardado en ejecutarse.

In [21]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
60.4 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.45 ms ± 363 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



El uso de matrices de tipo ``object`` también significa que si se realizan agregaciones como ``sum()`` o ``min()`` en una matriz con un valor ``None``, generalmente se obtendrá un error:

In [26]:
vals1.mean()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

Esto falla porque, como se puede ver en el error, la suma entre datos ``int`` y ``None`` no está definida.

### ``NaN``: Datos numéricos faltantes

La otra representación de datos faltantes que hemos comentado, ``NaN`` (acrónimo del inglés *Not a Number*), es diferente, pues es un valor float especial reconocido por todos los sistemas que utilizan la representación de punto flotante estándar del IEEE:

In [27]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

Si nos fijamos bien, podemos observar que lo que ahora tenemos es un dato de tipo flotante para representar un valor nulo, por lo que la matriz de valores numéricos seguirá manteniendo el mismo tipo float, admitiendo operaciones rápidas introducidas en código compilado, ya que no es necesario realizar una conversión a tipo ``object``.

Por otra parte, también deberemos tener claro que la naturaleza del valor ``NaN`` le hacen actuar como si fuera un virus, pues la mayoría de operaciones aritméticas en las que intervenga terminarán ofreciendo como resultado otro ``NaN``, independientemente del otro elemento de la operación.

In [28]:
1 + np.nan

nan

In [29]:
0 *  np.nan

nan

Además, esto también afecta a los agregados, donde un solo valor ``NaN`` hará que el resultado sea ``NaN``. Ojo, no dará error, por lo que no nos daremos cuenta del resultado salvo que analicemos el resultado o los valores que estamos agregando. Esto es muy importante porque puede que el resto de valores estén bien definidos, pero con que haya uno que sea ``NaN``, ya estaremos viciando el resultado:

In [30]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

Sin embargo, nuestros amigos de NumPy también han pensado en esto, por lo que nos ofrecen algunas agregaciones especiales que pueden lidiar con estas problemáticas (ignorando los valores nulos):

In [32]:
vals2

array([ 1., nan,  3.,  4.])

In [31]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

**IMPORTANTE**: El valor ``NaN`` es muy guay y nos ayuda en muchos casos pero ES un valor de TIPO FLOAT. Esto quiere decir que no es ni ``int`` ni ``string`` ni cualquier otro tipo de datos visto que no sea ``float``.

### NaN y None en Pandas

Los valores ``NaN`` y ``None`` son interesantes en función del caso que se nos plantee, no podemos decir que uno siempre es mejor que el otro. Por ello, Pandas lo ha tenido en cuenta para tratar con ellos de forma intercambiable, realizando las conversiones cuando sea apropiado:

In [37]:
a = pd.Series([1, np.nan, 2, None])

Para los tipos que no tienen disponible ningún valor centinela, Pandas casteará (convertirá) al tipo adecuado automáticamente los valores cuando detecte nulos.

Por ejemplo, si tenemos una matriz de enteros (int) y metemos un valor ``np.nan``, Pandas auto-convertirá los valores enteros de la matriz a punto flotante (float) para acomodar a nuestro nuevo compi:

In [38]:
# Series de tipo entero:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [39]:
# Establecemos un valor como None (que en Python hemos visto que es tipo object)
x[0] = None
x

0    NaN
1    1.0
dtype: float64

In [40]:
# Series de tipo string:
y = pd.Series(['a', 'b', 'c'])
y

0    a
1    b
2    c
dtype: object

In [43]:
# Establecemos un valor como None (que en Python hemos visto que es tipo object)
y[1] = np.nan
y

0    None
1     NaN
2       c
dtype: object

In [48]:
# Series de tipo boolean:
z = pd.Series([True, False, True])
z

0     True
1    False
2     True
dtype: bool

In [49]:
# Establecemos un valor como None (que en Python hemos visto que es tipo object)
z[0] = None
z

0    False
1    False
2     True
dtype: bool

Date cuenta de que además de castear los enteros a flotantes, Pandas también convierte el valor ``None`` (tipo object) a ``NaN`` (tipo float), que se amolda mejor a la matriz numérica.

La siguiente tabla recoge las convenciones de casteo de Pandas cuando se trata con valores nulos:

|Tipo          | Conversión al almacenar nulos  | Valor centinela (nulo)    |
|--------------|--------------------------------|---------------------------|
| ``floating`` | No change                      | ``np.nan``                |
| ``object``   | No change                      | ``None`` or ``np.nan``    |
| ``integer``  | Cast to ``float64``            | ``np.nan``                |
| ``boolean``  | Cast to ``bool``             | ``None`` or ``np.nan``    |

Recuerda que, en Pandas, los ``string`` se almacenan como ``object``.

## Operando con valores nulos

Tal como hemos visto, Pandas trata los valores ``None`` y ``NaN`` como elementos intercambiables para indicar la falta de datos.

Para facilitar esta convención, hay diversos métodos para detectar, reemplazar o eliminar estos valores nulos en los objetos de Pandas, es decir, en nuestros ``Series`` y ``DataFrames``:

- ``isnull()``: Nos devuelve una máscara booleana que nos indica las dónde hay valores nulos
- ``notnull()``: Devuelve la máscara booleana opuesta a la obtenida con ``isnull()``
- ``dropna()``: Nos devuelve el objeto de Pandas eliminando aquellos registros con valores nulos
- ``fillna()``: Devuelve una copia del objeto original reemplazando los valores nulos por lo que le especifiquemos


Con estas funciones podremos tratar a grandes rasgos todo lo relacionado con los nulos. Veamos cómo:

### Detectando valores nulos

Acabamos de ver 2 métodos para detectar nulos en ``Series`` o ``DataFrames``: ``isnull()`` y ``notnull()``, cuyo resultado será una máscara booleana que me dirá dónde hay nulos o donde no los hay, respectivamente:

In [88]:
import pandas as pd
import numpy as np 
data = pd.Series([1, np.nan, 'hello', 'None'])

In [58]:
data.isnull()

0    False
1     True
2    False
3    False
dtype: bool

In [59]:
data.notnull()

0     True
1    False
2     True
3     True
dtype: bool

Como podemos observar, los resultados son completamente opuestos, se puede obtener una a partir de negar la otra.


¿Y para qué se suele utilizar esto? Pues las opciones son múltiples, por ejemplo, hacer un filtrado en función de esta máscara. Vamos a hacerlo para los datos no nulos del ``Series`` ``data`` que hemos definido anteriormente:

In [63]:
data[data.notnull()]

0        1
2    hello
3     None
dtype: object

Del mismo modo que hemos visto para un ``Series``, también funcionarán para un ``DataFrame``, donde lo que nos devolverá será una matriz del mismo tamaño que la de valores del ``DataFrame``, donde cada uno de los elementos indicará la presencia (o no) de elementos nulos.

También podemos utilizar tanto ``isnull()`` como ``notnull()``, dependiendo de lo que queremos representar como ``True``:

In [64]:
df = pd.DataFrame([[np.nan, 2, 3], [np.nan, 1, np.nan], [np.nan, np.nan, np.nan], [np.nan, 2, 4], [np.nan, 2, 4]])
df

,0,1,2
0,NaN,2.0,3.0
1,NaN,1.0,NaN
2,NaN,NaN,NaN
3,NaN,2.0,4.0
4,NaN,2.0,4.0


In [76]:
df[df.notnull()]

,0,1,2
0,NaN,2.0,3.0
1,NaN,1.0,NaN
2,NaN,NaN,NaN
3,NaN,2.0,4.0
4,NaN,2.0,4.0


### Eliminando valores nulos

Además de los métodos de 'masking' que acabamos de ver, también hemos comentado que existen métodos de conveniencia: para quitar nulos (``dropna()``) y para sustituirlos (``fillna()``).


Para un objeto ``Series``, el funcionamiento es bastante sencillo, como vemos en el siguiente ejemplo sobre ``data``:

In [89]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [90]:
data.dropna()

0        1
2    hello
3     None
dtype: object

In [86]:
# data.dropna(inplace=True)

In [81]:
# data = data.dropna()

0        1
2    hello
3     None
dtype: object

Como podemos ver, eliminamos los valores nulos (fíjate además que el indice no se ha recalculado).


Para un ``DataFrame``, la cosa cambia, pues existen más opciones. Veámoslo con detenimiento, partiendo del siguiente ``DataFrame``:

In [91]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Evidentemente, en un ``DataFrame`` no podemos eliminar posiciones de la matriz, únicmente podremos eliminar filas o columnas completas. Qué eliminar dependerá del caso en el que nos encotnremos, por ello el método ``dropna()`` define cierto nivel de posibles cnsideraciones cuando tratamos con ``DataFrame``.


Por defecto, ``dropna()`` eliminará todas las filas en las que haya cualquier (any) valor nulo, aunque solo sea 1:

In [96]:
df.dropna(axis=0, how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [97]:
df2 = pd.DataFrame([[np.nan, 2, 3], [np.nan, 1, np.nan], [np.nan, np.nan, np.nan], [np.nan, 2, 4], [np.nan, 2, 4]])
df2

,0,1,2
0,NaN,2.0,3.0
1,NaN,1.0,NaN
2,NaN,NaN,NaN
3,NaN,2.0,4.0
4,NaN,2.0,4.0


In [99]:
df2.dropna(axis=1, how='all')

,1,2
0,2.0,3.0
1,1.0,NaN
2,NaN,NaN
3,2.0,4.0
4,2.0,4.0


In [ ]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Del mismo modo a lo que hemos visto en *notebooks* anteriores, muchos métodos y funciones que trabajan sobre ``DataFrames`` nos permiten trabajar en uno u otro eje, es decir, a nivel fila o a nivel columna, lo que podemos conseguir con el parámetro ``axis``, donde ``axis=1`` (o ``axis='columns'``) eliminará las columnas que tengan al menos 1 nulo:

In [101]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


Esto nos puede ser de ayuda en muchos casos, sin embargo, al eliminar todo lo que tenga al menos un nulo, estamos desechando el resto de datos, que podrían ser de utilidad. Por eso surge la necesidad de poder variar ese umbral de nulos para eliminr la fila o columna, lo cual podemos hacer mediante los parámetros ``how`` y ``thresh``.



Si nos centramos en el primero (``how``), podremos utilizarlo para decidir si eliminamos una fila o columna en función de si tiene al menos un nulo (``how='any'``, que es el valor por defecto) o si tiene todos sus valores nulos(``how='all'``):

In [102]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [103]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


El segundo parámetro, ``thresh``, nos permitirá realizar esta distinción hilando más fino, pues no permitirá imponer un umbral a partir del cual eliminaremos la fila. Este umbral hará referencia al mínimo número de elementos NO nulos para que una fila o columna no sea eliminada. Por ejemplo, si queremos quedarnos con todas las filas que tengan al menos 3 valores no nulos:

In [110]:
df.loc[0, 2] = np.nan
df

,0,1,2,3
0,1.0,NaN,NaN,NaN
1,2.0,3.0,5.0,NaN
2,NaN,4.0,6.0,NaN


In [121]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5.0,NaN


En este caso, solo pasaría nuestro filtro la segunda fila (``index=1``).

### Reemplazando valores nulos

A veces, en lugar de eliminar los valores nulos, nos puede interesar más reemplazarlos con algún valor válido y así poder seguir operando con esa fila o columna.

Este valor podría ser un valor único, como el ``0``, o podríamos realizar algún tipo de imputación o interpolación de los valores 'buenos'.
Podríamos hacer esto mediante el método ``isnull()`` utilizando la máscara que nos devuelve. Sin embargo, esto supondría cierta complejidad, cosa que Pandas nos facilita con otro método: ``fillna()``, que devolverá una copia de la matriz con los valores nulos reemplazados por lo que le pasemos como argumento.

Veamos un ejemplo con el siguiente ``Series``:

In [123]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Podemos reemplazar los nulos con algún valor en concreto, como el 0 (muy típico):

In [124]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Sin embargo, rellenar con un único valor no es la única forma de rellenar estos nulos, también podemos utilizar ciertas técnicas como la sustitución ``forward-fill``, que, cuando ve un nulo, rellena con el valor anterior:

In [126]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Del mismo modo, existe una función análoga que rellena los valores nulos con el valor siguiente, propagándolo hacia atrás en el llamado ``back-fill``:

In [127]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Si bien acabamos de ver estos ejemplo aplicados sobre un objeto ``Series``, también podemos aplicarlo sobre un ``DataFrame``, pudiendo especificar ese segundo grado de libertad que nos aporta el parámetro ``axis``, para filas o columnas:

In [128]:
df

,0,1,2,3
0,1.0,NaN,NaN,NaN
1,2.0,3.0,5.0,NaN
2,NaN,4.0,6.0,NaN


In [132]:
df.fillna(method='ffill', axis = 1).fillna(method='bfill', axis = 1)

,0,1,2,3
0,1.0,1.0,1.0,1.0
1,2.0,3.0,5.0,5.0
2,4.0,4.0,6.0,6.0


¡OJO! Estos dos últimos métodos no son infalibles, ya que nos podemos encontrar con situaciones donde no se pueda propagar un valor bueno:

In [133]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [134]:
df.fillna(method='ffill', axis = 'rows')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,2.0,4.0,6


In [135]:
df.fillna(method='bfill', axis = 'rows')

,0,1,2
0,1.0,3.0,2
1,2.0,3.0,5
2,NaN,4.0,6


# Algunos ejemplos prácticos

In [136]:
import pandas as pd
import numpy as np

In [137]:
dummy_df = pd.read_csv('dummy_data.csv')

In [138]:
dummy_df

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
3,4,Sophia,NaN,143.0
4,5,Bob,NaN,NaN
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0
7,8,Will,29.0,NaN
8,9,Scott,NaN,148.0


# Razones de la falta de datos

1. El usuario no quiso rellenar algunos datos por razones de privacidad
2. Pérdida de datos al transferir los datos o que parte de ellos se haya corrompido en la base de datos
3. Datos insuficientes para rellenar una columna que dependa de otros, etc.

Pero puede haber muchos motivos más para encontrarnos datos faltantes en un dataset.

## df.describe()

Una de las herramientas que hemos visto anteriormente es el método ``describe()``, que nos aporta ciertos estadísticos de utilidad de nuestro dataset, cosa que ahora nos puede ser de mucha utilidad, sobre todo, el conteo, que nos dirá cuántos valores no nulos tenemos:

In [139]:
dummy_df.describe()

,Sno,Age,Height(cm)
count,9.000000,6.000000,7.000000
mean,5.000000,28.166667,154.857143
std,2.738613,2.316607,7.174691
min,1.000000,25.000000,143.000000
25%,3.000000,26.500000,151.000000
50%,5.000000,28.500000,156.000000
75%,7.000000,29.750000,160.000000
max,9.000000,31.000000,163.000000


Para contrastar con el número de filas totales, podemos utilizar alguna de las formas que hemos visto en los primeros *notebooks*, como:

In [140]:
dummy_df.shape[0]

9

## Ejemplo con strings

Recordemos que el valor ``NaN`` es de tipo float, pero en este caso lo estaremos combinando con ``strings``, que Pandas interpreta como ``object``. Esto no supone ningún problema pues el array seguirá siendo de tipo ``object``:

In [141]:
string_dummy_df = pd.read_csv('dummy_str_data.csv',index_col=0)
string_dummy_df

,Device_name,Device_description,Single-Use
Sno,,,
1,Synringe,Used to inject medicine,True
2,Ventilator,Used to help patients breath,False
3,Surgical Gloves,NaN,True
4,Stethescopes,NaN,NaN
5,Vials container,NaN,NaN


Veamos cómo aplicar algunos de los métodos que ya hemos visto.

## df.isna() and df.notna()

Así como hemos visto los métodos ``isnull()`` y ``notnull()``, tenemos los métodos ``isna()`` y ``notna()``, que nos devuelven lo mismo, las matrices booleanas que indican si un elemento es nulo o no, dependiendo del método que utilicemos:

In [142]:
dummy_df.isna()

,Sno,Name,Age,Height(cm)
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,True,False
4,False,False,True,True
5,False,False,False,False
6,False,False,False,False
7,False,False,False,True
8,False,False,True,False


In [143]:
dummy_df.notna()

,Sno,Name,Age,Height(cm)
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True
3,True,True,False,True
4,True,True,False,False
5,True,True,True,True
6,True,True,True,True
7,True,True,True,False
8,True,True,False,True


## df.isnull()

In [144]:
dummy_df.isnull()

,Sno,Name,Age,Height(cm)
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,True,False
4,False,False,True,True
5,False,False,False,False
6,False,False,False,False
7,False,False,False,True
8,False,False,True,False


## Conteo de los nulos en cada columna

Así como hemos visto que con el ``describe()`` podríamos obtener los valores no nulos, también podemos replicar esta medida con los métods que acabamos de ver (lo qu enos permitirá contar los nulos o los no nulos). Por ejemplo, podríamos hacer:

In [154]:
dummy_df.isnull().sum(axis=0)

Sno           0
Name          0
Age           3
Height(cm)    2
dtype: int64

### A veces no sabremos cómo rellenar un valor nulo en función del resto de valores de la columna

Como pasa en el caso de 'Device_description':

In [157]:
string_dummy_df = pd.read_csv('dummy_str_data.csv', index_col=0)

In [158]:
string_dummy_df

,Device_name,Device_description,Single-Use
Sno,,,
1,Synringe,Used to inject medicine,True
2,Ventilator,Used to help patients breath,False
3,Surgical Gloves,NaN,True
4,Stethescopes,NaN,NaN
5,Vials container,NaN,NaN


Por lo tanto, no podremos utilizar los métods de imputación, así que tendríamos que pensar mejor la estrategia a seguir.

# Datos faltantes en fechas

Así como hemos visto el efecto de los nulos en matrices de ``strings`` y ``enteros``, también nos peude pasar con los datos de fechas:

In [159]:
time_df = pd.read_csv('dummy_time.csv', index_col=0)

In [116]:
time_df

,Name,Age,Height(cm),birthday
Sno,,,,
1,John,25.0,160.0,1994-01-01
2,Jimmy,26.0,163.0,NaN
3,Felicia,28.0,154.0,1995-01-01
4,Sophia,NaN,143.0,NaN
5,Bob,NaN,NaN,1994-01-01
6,Billy,30.0,156.0,1994-01-01
7,Kate,31.0,160.0,1990-01-01
8,Will,29.0,NaN,1991-07-01
9,Scott,NaN,148.0,NaN


In [161]:
type(time_df['birthday'].iloc[0])

str

In [162]:
time_df['birthday'] = pd.to_datetime(time_df['birthday'])

In [166]:
time_df

,Name,Age,Height(cm),birthday
Sno,,,,
1,John,25.0,160.0,1994-01-01
2,Jimmy,26.0,163.0,NaT
3,Felicia,28.0,154.0,1995-01-01
4,Sophia,NaN,143.0,NaT
5,Bob,NaN,NaN,1994-01-01
6,Billy,30.0,156.0,1994-01-01
7,Kate,31.0,160.0,1990-01-01
8,Will,29.0,NaN,1991-07-01
9,Scott,NaN,148.0,NaT


Como te pudes fijar, en el caso de los nulos para fechas, el valor rompe con lo vistoa nteriormente, pues no es ni ``None`` ni ``NaN``, sino ``NaT (Not a Time)``, cosa que no pasaba antes.

# Estrategias para trabajar con datos faltantes

## 1. Ignorando los datos nulos

### 1.1 Eliminando las filas con nulos

In [167]:
dummy_df = pd.read_csv('dummy_data.csv', index_col=0)

In [168]:
dummy_df

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
3,4,Sophia,NaN,143.0
4,5,Bob,NaN,NaN
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0
7,8,Will,29.0,NaN
8,9,Scott,NaN,148.0


In [169]:
removed_na_df = dummy_df.dropna()
removed_na_df

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0


En este caso, dado que estamos eliminando 4 de 9 filas que teníamos (44,44%), no parece que sea la mejor solución ya que perdemremos muchos datos.

### 1.2 Eliminando filas con la mayoría de columnas nulas

In [170]:
dummy_majority_df = pd.read_csv('dummy_missing_majority.csv', index_col=0)

In [171]:
dummy_majority_df

,Name,Age,Height(cm),Marks(100),Country,City
Sno,,,,,,
1,John,25.0,160.0,80.0,USA,New York
2,Jimmy,26.0,163.0,NaN,UK,London
3,Felicia,28.0,154.0,NaN,USA,Miami
4,Sophia,NaN,143.0,NaN,NaN,NaN
5,Bob,NaN,NaN,NaN,NaN,NaN
6,Billy,30.0,156.0,NaN,France,Paris
7,Kate,31.0,160.0,NaN,Italy,Rome
8,Will,29.0,NaN,NaN,Russia,Moscow
9,Scott,NaN,148.0,NaN,NaN,NaN


Para ello, hemos visto el parámetro `thresh`, con el que le podemos decir al método ``dropna()`` que mantenga las filas que al menos tengan x filas no nulas:

In [172]:
dummy_majority_df.dropna(thresh=4)

,Name,Age,Height(cm),Marks(100),Country,City
Sno,,,,,,
1,John,25.0,160.0,80.0,USA,New York
2,Jimmy,26.0,163.0,NaN,UK,London
3,Felicia,28.0,154.0,NaN,USA,Miami
6,Billy,30.0,156.0,NaN,France,Paris
7,Kate,31.0,160.0,NaN,Italy,Rome
8,Will,29.0,NaN,NaN,Russia,Moscow


In [173]:
dummy_majority_df.dropna(thresh=6)

,Name,Age,Height(cm),Marks(100),Country,City
Sno,,,,,,
1,John,25.0,160.0,80.0,USA,New York


In [174]:
dummy_majority_df.dropna()

,Name,Age,Height(cm),Marks(100),Country,City
Sno,,,,,,
1,John,25.0,160.0,80.0,USA,New York


### 1.3 Eliminando columnas por porcentaje de vacíos

Basándose en lo visto anteriormente, podremos descartar aquellas columnas con un porcentaje de nulos mayor o igual al que queremos, que en este caso lo pondremos en el 40%:

In [176]:
dummy_majority_df

,Name,Age,Height(cm),Marks(100),Country,City
Sno,,,,,,
1,John,25.0,160.0,80.0,USA,New York
2,Jimmy,26.0,163.0,NaN,UK,London
3,Felicia,28.0,154.0,NaN,USA,Miami
4,Sophia,NaN,143.0,NaN,NaN,NaN
5,Bob,NaN,NaN,NaN,NaN,NaN
6,Billy,30.0,156.0,NaN,France,Paris
7,Kate,31.0,160.0,NaN,Italy,Rome
8,Will,29.0,NaN,NaN,Russia,Moscow
9,Scott,NaN,148.0,NaN,NaN,NaN


In [175]:
dummy_majority_df.dropna(axis=1, thresh=int(0.4*len(dummy_majority_df)))

,Name,Age,Height(cm),Country,City
Sno,,,,,
1,John,25.0,160.0,USA,New York
2,Jimmy,26.0,163.0,UK,London
3,Felicia,28.0,154.0,USA,Miami
4,Sophia,NaN,143.0,NaN,NaN
5,Bob,NaN,NaN,NaN,NaN
6,Billy,30.0,156.0,France,Paris
7,Kate,31.0,160.0,Italy,Rome
8,Will,29.0,NaN,Russia,Moscow
9,Scott,NaN,148.0,NaN,NaN


Como puedes observar, se ha eliminado la columna ``Marks``, ya que tenía más de un 40% de nulos

### 1.4 Eliminando filas con la mayoría de columnas nulas

En este caso, mantendremos las filas que tengan, al menos, un 60% de valores no nulos:

In [178]:
dummy_majority_df.dropna(axis=0, thresh=int(0.6*len(dummy_majority_df.columns)))

,Name,Age,Height(cm),Marks(100),Country,City
Sno,,,,,,
1,John,25.0,160.0,80.0,USA,New York
2,Jimmy,26.0,163.0,NaN,UK,London
3,Felicia,28.0,154.0,NaN,USA,Miami
6,Billy,30.0,156.0,NaN,France,Paris
7,Kate,31.0,160.0,NaN,Italy,Rome
8,Will,29.0,NaN,NaN,Russia,Moscow


## 2. Imputando valores

### 2.1 Rellenando con valores genéricos

In [179]:
dummy_df = pd.read_csv('dummy_data.csv', index_col=0)
dummy_df

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
3,4,Sophia,NaN,143.0
4,5,Bob,NaN,NaN
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0
7,8,Will,29.0,NaN
8,9,Scott,NaN,148.0


In [180]:
dummy_df.fillna(-1)

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
3,4,Sophia,-1.0,143.0
4,5,Bob,-1.0,-1.0
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0
7,8,Will,29.0,-1.0
8,9,Scott,-1.0,148.0


**IMPORTANTE**: Los cambios no se guardarán en el dataframe original a no ser que se lo especifiquemos: o bien asignando esta salida a la variable del ``DataFrame`` original, o bien utilizando el parámetro ``inplace`` y estableciéndolo como ``True``.

In [142]:
# dummy_df.fillna(inplace=True)

Además, así como hemos visto los métodos de imputación mediante los parámetros donde especificábamos el ``how``, podemos realizarlo con un método particular, tanto para 'back-fill' como 'forward-fill':

In [181]:
dummy_df.bfill()

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
3,4,Sophia,30.0,143.0
4,5,Bob,30.0,156.0
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0
7,8,Will,29.0,148.0
8,9,Scott,NaN,148.0


In [182]:
dummy_df.ffill()

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
3,4,Sophia,28.0,143.0
4,5,Bob,28.0,143.0
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0
7,8,Will,29.0,160.0
8,9,Scott,29.0,148.0


### 2.2 Rellenando con las tendencias centrales

In [183]:
dummy_df

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
3,4,Sophia,NaN,143.0
4,5,Bob,NaN,NaN
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0
7,8,Will,29.0,NaN
8,9,Scott,NaN,148.0


In [190]:
mean_age = dummy_df['Age'].mean()
mean_height = dummy_df['Height(cm)'].mean()

In [191]:
map_dict = {'Age': mean_age, 'Height(cm)': mean_height}
map_dict

{'Age': 28.166666666666668, 'Height(cm)': 154.85714285714286}

In [192]:
dummy_df.fillna(value=map_dict)

,Sno,Name,Age,Height(cm)
0,1,John,25.000000,160.000000
1,2,Jimmy,26.000000,163.000000
2,3,Felicia,28.000000,154.000000
3,4,Sophia,28.166667,143.000000
4,5,Bob,28.166667,154.857143
5,6,Billy,30.000000,156.000000
6,7,Kate,31.000000,160.000000
7,8,Will,29.000000,154.857143
8,9,Scott,28.166667,148.000000


### 2.3 Imputando valores basados en condición

In [193]:
weight_df = pd.read_csv('dummy_age_weight.csv')

In [194]:
weight_df

,Gender,Weight(kg)
0,Male,70.0
1,Female,55.0
2,Male,65.0
3,Female,NaN
4,Female,60.0
5,Male,NaN
6,Female,52.0
7,Female,53.0
8,Male,85.0
9,Male,75.0


In [197]:
weight_df["Weight(kg)"] = weight_df.groupby("Gender").transform(lambda x: x.fillna(x.mean()))

In [198]:
weight_df

,Gender,Weight(kg)
0,Male,70.00
1,Female,55.00
2,Male,65.00
3,Female,57.60
4,Female,60.00
5,Male,73.75
6,Female,52.00
7,Female,53.00
8,Male,85.00
9,Male,75.00


De este modo, estamos rellenando los nulos con un valor diferente en función del grupo al que pertenezcan:

In [199]:
weight_df.groupby("Gender")

In [203]:
for x in weight_df.groupby("Gender"):
    print("Printing Group")
    print(x)
    print(x[1].mean())
    print("\n\n")

Printing Group
('Female',     Gender  Weight(kg)
1   Female        55.0
3   Female        57.6
4   Female        60.0
6   Female        52.0
7   Female        53.0
11  Female        68.0)
Weight(kg)    57.6
dtype: float64



Printing Group
('Male',    Gender  Weight(kg)
0    Male       70.00
2    Male       65.00
5    Male       73.75
8    Male       85.00
9    Male       75.00
10   Male       73.75)
Weight(kg)    73.75
dtype: float64



